# Contextual Chunk Headers (CCH)

## Overview

Contextual chunk headers (CCH) is a method of creating chunk headers that contain higher-level context (such as document-level or section-level context), and prepending those chunk headers to the chunks prior to embedding them. This gives the embeddings a much more accurate and complete representation of the content and meaning of the text. In our testing, this feature leads to a substantial improvement in retrieval quality. In addition to increasing the rate at which the correct information is retrieved, CCH also reduces the rate at which irrelevant results show up in the search results. This reduces the rate at which the LLM misinterprets a piece of text in downstream chat and generation applications.

## Motivation

Many of the problems developers face with RAG come down to this: Individual chunks oftentimes do not contain sufficient context to be properly used by the retrieval system or the LLM. This leads to the inability to answer questions and, more worryingly, hallucinations.

Examples of this problem
- Chunks oftentimes refer to their subject via implicit references and pronouns. This causes them to not be retrieved when they should be, or to not be properly understood by the LLM.
- Individual chunks oftentimes only make sense in the context of the entire section or document, and can be misleading when read on their own.

## Key Components

#### Contextual chunk headers
The idea here is to add in higher-level context to the chunk by prepending a chunk header. This chunk header could be as simple as just the document title, or it could use a combination of document title, a concise document summary, and the full hierarchy of section and sub-section titles.

## Method Details

#### Context generation
In the demonstration below we use an LLM to generate a descriptive title for the document. This is done through a simple prompt where you pass in a truncated version of the document text and ask the LLM to generate a descriptive title for the document. If you already have sufficiently descriptive document titles then you can directly use those instead. We've found that a document title is the simplest and most important kind of higher-level context to include in the chunk header.

Other kinds of context you can include in the chunk header:
- Concise document summary
- Section/sub-section title(s)
    - This helps the retrieval system handle queries for larger sections or topics in documents.

#### Embed chunks with chunk headers
The text you embed for each chunk is simply the concatenation of the chunk header and the chunk text. If you use a reranker during retrieval, you'll want to make sure you use this same concatenation there too.

#### Add chunk headers to search results
Including the chunk headers when presenting the search results to the LLM is also beneficial as it gives the LLM more context, and makes it less likely that it misunderstands the meaning of a chunk.

![Your Technique Name](https://github.com/NirDiamant/RAG_Techniques/blob/main/images/contextual_chunk_headers.svg?raw=1)

## Setup

You'll need a Cohere API key and an OpenAI API key for this notebook.

# Package Installation and Imports

The cell below installs all necessary packages required to run this notebook.


In [ ]:
# Install required packages
!pip install -q langchain openai python-dotenv tiktoken langchain_groq cohere

In [2]:
!pip install -q langchain_text_splitters

In [35]:
import cohere
import tiktoken
from typing import List
from openai import OpenAI
import os
from dotenv import load_dotenv
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load environment variables from a .env file
load_dotenv()
os.environ["CO_API_KEY"] = "" # Cohere API key
os.environ["GROQ_API_KEY"] = "" # OpenAI API key

## Load the document and split it into chunks
We'll use the basic LangChain RecursiveCharacterTextSplitter for this demo, but you can combine CCH with more sophisticated chunking methods for even better performance.

In [4]:
# Download required data files
import os
os.makedirs('data', exist_ok=True)

# Download the PDF document used in this notebook
!wget -O data/Understanding_Climate_Change.pdf https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf
!wget -O data/nike_2023_annual_report.txt https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/nike_2023_annual_report.txt


--2026-01-22 02:27:45--  https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 206372 (202K) [application/octet-stream]
Saving to: ‘data/Understanding_Climate_Change.pdf’

data/Understanding_ 100%[===================>] 201.54K  --.-KB/s    in 0.01s   

2026-01-22 02:27:46 (13.9 MB/s) - ‘data/Understanding_Climate_Change.pdf’ saved [206372/206372]

--2026-01-22 02:27:46--  https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/nike_2023_annual_report.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

def split_into_chunks(text: str, chunk_size: int = 800) -> list[str]:
    """
    Split a given text into chunks of specified size using RecursiveCharacterTextSplitter.

    Args:
        text (str): The input text to be split into chunks.
        chunk_size (int, optional): The maximum size of each chunk. Defaults to 800.

    Returns:
        list[str]: A list of text chunks.

    Example:
        >>> text = "This is a sample text to be split into chunks."
        >>> chunks = split_into_chunks(text, chunk_size=10)
        >>> print(chunks)
        ['This is a', 'sample', 'text to', 'be split', 'into', 'chunks.']
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=0,
        length_function=len
    )
    documents = text_splitter.create_documents([text])
    return [document.page_content for document in documents]

# File path for the input document
FILE_PATH = "data/nike_2023_annual_report.txt"

# Read the document and split it into chunks
with open(FILE_PATH, "r") as file:
    document_text = file.read()

chunks = split_into_chunks(document_text, chunk_size=800)

## Generate descriptive document title to use in chunk header

In [7]:
# Constants
DOCUMENT_TITLE_PROMPT = """
INSTRUCTIONS
What is the title of the following document?

Your response MUST be the title of the document, and nothing else. DO NOT respond with anything else.

{document_title_guidance}

{truncation_message}

DOCUMENT
{document_text}
""".strip()

TRUNCATION_MESSAGE = """
Also note that the document text provided below is just the first ~{num_words} words of the document. That should be plenty for this task. Your response should still pertain to the entire document, not just the text provided below.
""".strip()

In [10]:
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

In [11]:
MAX_CONTENT_TOKENS = 4000
MODEL_NAME = "llama-3.3-70b-versatile"

def make_llm_call(chat_messages: list[dict]) -> str:
    """
    Make an API call using ChatGroq (LangChain wrapper).
    """

    llm = ChatGroq(
        groq_api_key=os.environ["GROQ_API_KEY"],
        model_name=MODEL_NAME,
        temperature=0.2,
        max_tokens=MAX_CONTENT_TOKENS,
    )

    # Convert OpenAI-style messages -> LangChain messages
    lc_messages = []
    for msg in chat_messages:
        role = msg["role"]
        content = msg["content"]

        if role == "system":
            lc_messages.append(SystemMessage(content=content))
        elif role == "user":
            lc_messages.append(HumanMessage(content=content))
        elif role == "assistant":
            lc_messages.append(AIMessage(content=content))

    response = llm.invoke(lc_messages)
    return response.content.strip()


In [12]:
# test
chat_messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant."
    },
    {
        "role": "user",
        "content": "Explain what Retrieval-Augmented Generation (RAG) is in simple terms."
    }
]

response = make_llm_call(chat_messages)
print(response)

Retrieval-Augmented Generation (RAG) is a technique used in artificial intelligence, particularly in natural language processing. In simple terms, RAG is a way for AI models to generate text by combining two main steps:

1. **Retrieval**: The AI model searches a large database or knowledge base to find relevant information related to the topic or question it's trying to answer.
2. **Generation**: The AI model uses the retrieved information to generate a response, such as a paragraph or a sentence.

Think of it like a researcher writing a paper. First, they search for relevant information in books and articles (retrieval). Then, they use that information to write their own text (generation). RAG works in a similar way, but instead of a human researcher, it's an AI model that's doing the searching and writing.

The goal of RAG is to improve the accuracy and coherence of generated text by providing the AI model with relevant context and information. This can be especially useful for tasks

In [16]:
def truncate_content(
    text: str,
    max_tokens: int = 400,
    chars_per_token: float = 4.0,
) -> str:
    """
    Truncate text approximately by tokens without using OpenAI tokenizer.

    Args:
        text (str): Input text
        max_tokens (int): Max token budget
        chars_per_token (float): Approximation (English ~4, Vietnamese ~3–4)

    Returns:
        str: Truncated text
    """
    max_chars = int(max_tokens * chars_per_token)

    if len(text) <= max_chars:
        return text

    return text[:max_chars]

In [17]:
truncate_content("""Retrieval-Augmented Generation (RAG) is a technique used in artificial intelligence, particularly in natural language processing. In simple terms, RAG is a way for AI models to generate text by combining two main steps:

1. **Retrieval**: The AI model searches a large database or knowledge base to find relevant information related to the topic or question it's trying to answer.
2. **Generation**: The AI model uses the retrieved information to generate a response, such as a paragraph or a sentence.

Think of it like a researcher writing a paper. First, they search for relevant information in books and articles (retrieval). Then, they use that information to write their own text (generation). RAG works in a similar way, but instead of a human researcher, it's an AI model that's doing the searching and writing.

The goal of RAG is to improve the accuracy and coherence of generated text by providing the AI model with relevant context and information. This can be especially useful for tasks like answering questions, summarizing documents, or generating creative content.""")

"Retrieval-Augmented Generation (RAG) is a technique used in artificial intelligence, particularly in natural language processing. In simple terms, RAG is a way for AI models to generate text by combining two main steps:\n\n1. **Retrieval**: The AI model searches a large database or knowledge base to find relevant information related to the topic or question it's trying to answer.\n2. **Generation**: The AI model uses the retrieved information to generate a response, such as a paragraph or a sentence.\n\nThink of it like a researcher writing a paper. First, they search for relevant information in books and articles (retrieval). Then, they use that information to write their own text (generation). RAG works in a similar way, but instead of a human researcher, it's an AI model that's doing the searching and writing.\n\nThe goal of RAG is to improve the accuracy and coherence of generated text by providing the AI model with relevant context and information. This can be especially useful f

In [31]:
def get_document_title(
    text: str,
    llm,
    max_chars: int = 1500
) -> str:
    """
    Generate a concise document title using LLM (ChatGroq).
    """

    if not text or not text.strip():
        return "Untitled Document"

    text = text.strip()[:max_chars]

    messages = [
        {
            "role": "system",
            "content": (
                "You are an assistant that generates concise, informative document titles. "
                "Return ONLY the title. No explanations, no punctuation at the end."
            )
        },
        {
            "role": "user",
            "content": (
                "Generate a short title (max 10 words) for the following document:\n\n"
                f"{text}"
            )
        }
    ]

    try:
        title = make_llm_call(messages)
        return title.strip().strip('"')
    except Exception as e:
        return "Untitled Document"

# Example usage
if __name__ == "__main__":
    llm = ChatGroq(
        groq_api_key=os.environ["GROQ_API_KEY"],
        model_name=MODEL_NAME,
        temperature=0.0,
        max_tokens=MAX_CONTENT_TOKENS,
    )
    # Assuming document_text is defined elsewhere
    document_text_1 = """
Retrieval-Augmented Generation (RAG) is a technique used in artificial intelligence, particularly in natural language processing. In simple terms, RAG is a way for AI models to generate text by combining two main steps:

1. **Retrieval**: The AI model searches a large database or knowledge base to find relevant information related to the topic or question it's trying to answer.
2. **Generation**: The AI model uses the retrieved information to generate a response, such as a paragraph or a sentence.

Think of it like a researcher writing a paper. First, they search for relevant information in books and articles (retrieval). Then, they use that information to write their own text (generation). RAG works in a similar way, but instead of a human researcher, it's an AI model that's doing the searching and writing.

The goal of RAG is to improve the accuracy and coherence of generated text by providing the AI model with relevant context and information. This can be especially useful for tasks like answering questions, summarizing documents, or generating creative content.
    """
    document_title = get_document_title(document_text, llm)
    print(f"Document Title: {document_title}")

Document Title: Nike Inc Annual Report 2023


## Add chunk header and measure impact
Let's look at a specific example to demonstrate the impact of adding a chunk header. We'll use the Cohere reranker to measure relevance to a query with and without a chunk header.

In [28]:
def rerank_documents(query: str, chunks: List[str]) -> List[float]:
    """
    Use Cohere Rerank API to rerank the search results.

    Args:
        query (str): The search query.
        chunks (List[str]): List of document chunks to be reranked.

    Returns:
        List[float]: List of similarity scores for each chunk, in the original order.
    """
    MODEL = "rerank-multilingual-v3.0"
    client = cohere.Client(api_key=os.environ["CO_API_KEY"])

    reranked_results = client.rerank(model=MODEL, query=query, documents=chunks)
    results = reranked_results.results
    reranked_indices = [result.index for result in results]
    reranked_similarity_scores = [result.relevance_score for result in results]

    # Convert back to order of original documents
    similarity_scores = [0] * len(chunks)
    for i, index in enumerate(reranked_indices):
        similarity_scores[index] = reranked_similarity_scores[i]

    return similarity_scores

In [34]:
def compare_chunk_similarities(chunk_index: int, chunks: List[str], document_title: str, query: str) -> None:
    """
    Compare similarity scores for a chunk with and without a contextual header.

    Args:
        chunk_index (int): Index of the chunk to inspect.
        chunks (List[str]): List of all document chunks.
        document_title (str): Title of the document.
        query (str): The search query to use for comparison.

    Prints:
        Chunk header, chunk text, query, and similarity scores with and without the header.
    """
    chunk_text = chunks[chunk_index]
    chunk_wo_header = chunk_text
    chunk_w_header = f"Document Title: {document_title}\n\n{chunk_text}"

    similarity_scores = rerank_documents(query, [chunk_wo_header, chunk_w_header])

    print(f"\nChunk header:\nDocument Title: {document_title}")
    print(f"\nChunk text:\n{chunk_text}")
    print(f"\nQuery: {query}")
    print(f"\nSimilarity without contextual chunk header: {similarity_scores[0]:.4f}")
    print(f"Similarity with contextual chunk header: {similarity_scores[1]:.4f}")

# Notebook cell for execution
# Assuming chunks and document_title are defined in previous cells
CHUNK_INDEX_TO_INSPECT = 86
QUERY = "Nike climate change impact"

compare_chunk_similarities(CHUNK_INDEX_TO_INSPECT, chunks, document_title, QUERY)


Chunk header:
Document Title: Nike Inc Annual Report 2023

Chunk text:
Given the broad and global scope of our operations, we are particularly vulnerable to the physical risks of climate change, such 
as shifts in weather patterns. Extreme weather conditions in the areas in which our retail stores, suppliers, manufacturers, 
customers, distribution centers, offices, headquarters and vendors are located could adversely affect our operating results and 
financial condition. Moreover, natural disasters such as earthquakes, hurricanes, wildfires, tsunamis, floods or droughts, whether 
occurring in the United States or abroad, and their related consequences and effects, including energy shortages and public 
health issues, have in the past temporarily disrupted, and could in the future disrupt, our operations, the operations of our

Query: Nike climate change impact

Similarity without contextual chunk header: 0.0099
Similarity with contextual chunk header: 0.9983


This chunk is clearly about the impact of climate change on some organization, but it doesn't explicitly say "Nike" in it. So the relevance to the query "Nike climate change impact" in only about 0.1. By simply adding the document title to the chunk that similarity goes up to 0.92.

# Eval results

#### KITE

We evaluated CCH on an end-to-end RAG benchmark we created, called KITE (Knowledge-Intensive Task Evaluation).

KITE currently consists of 4 datasets and a total of 50 questions.
- **AI Papers** - ~100 academic papers about AI and RAG, downloaded from arXiv in PDF form.
- **BVP Cloud 10-Ks** - 10-Ks for all companies in the Bessemer Cloud Index (~70 of them), in PDF form.
- **Sourcegraph Company Handbook** - ~800 markdown files, with their original directory structure, downloaded from Sourcegraph's publicly accessible company handbook GitHub [page](https://github.com/sourcegraph/handbook/tree/main/content).
- **Supreme Court Opinions** - All Supreme Court opinions from Term Year 2022 (delivered from January '23 to June '23), downloaded from the official Supreme Court [website](https://www.supremecourt.gov/opinions/slipopinion/22) in PDF form.

Ground truth answers are included with each sample. Most samples also include grading rubrics. Grading is done on a scale of 0-10 for each question, with a strong LLM doing the grading.

We compare performance with and without CCH. For the CCH config we use document title and document summary. All other parameters remain the same between the two configurations. We use the Cohere 3 reranker, and we use GPT-4o for response generation.

|                         | No-CCH   | CCH          |
|-------------------------|----------|--------------|
| AI Papers               | 4.5      | 4.7          |
| BVP Cloud               | 2.6      | 6.3          |
| Sourcegraph             | 5.7      | 5.8          |
| Supreme Court Opinions  | 6.1      | 7.4          |
| **Average**             | 4.72     | 6.04         |

We can see that CCH leads to an improvement in performance on each of the four datasets. Some datasets see a large improvement while others see a small improvement. The overall average score increases from 4.72 -> 6.04, a 27.9% increase.

#### FinanceBench

We've also evaluated CCH on FinanceBench, where it contributed to a score of 83%, compared to a baseline score of 19%. For that benchmark, we tested CCH and relevant segment extraction (RSE) jointly, so we can't say exactly how much CCH contributed to that result. But the combination of CCH and RSE clearly leads to substantial accuracy improvements on FinanceBench.

![](https://europe-west1-rag-techniques-views-tracker.cloudfunctions.net/rag-techniques-tracker?notebook=all-rag-techniques--contextual-chunk-headers)